In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import precision_score, recall_score


class MNISTCSV(Dataset):
    def __init__(self, csv_file):
        data = pd.read_csv(csv_file)
        self.labels = torch.tensor(data.iloc[:, 0].values, dtype=torch.long)  
        self.images = torch.tensor(data.iloc[:, 1:].values, dtype=torch.float32) / 255.0  
        self.images = (self.images - 0.5) / 0.5  #

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx].view(1, 28, 28)  
        label = self.labels[idx]
        return image, label

train_dataset = MNISTCSV("/Users/nikhilsharma/Downloads/mnist_train.csv")
test_dataset = MNISTCSV("/Users/nikhilsharma/Downloads/mnist_test.csv")

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, num_workers=0)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False, num_workers=0)


class FNN(nn.Module): 
    def __init__(self): 
        super(FNN, self).__init__() 
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10) 
    
    def forward(self, x): 
        x = x.view(-1, 28 * 28) 
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x)) 
        x = self.fc3(x) 
        return x  


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 5
for epoch in range(num_epochs): 
    model.train()
    for images, labels in train_loader: 
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        output = model(images) 
        loss = criterion(output, labels) 
        loss.backward() 
        optimizer.step()  
        
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


model.eval()
all_labels = []
all_preds = []
with torch.no_grad(): 
    for images, labels in test_loader: 
        images, labels = images.to(device), labels.to(device)
        output = model(images) 
        _, predicted = torch.max(output.data, 1) 
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

accuracy = 100 * np.mean(np.array(all_labels) == np.array(all_preds))
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')

print(f"Test Accuracy: {accuracy:.2f}%")
print(f"Macro Precision: {precision:.4f}")
print(f"Macro Recall: {recall:.4f}")


Epoch [1/5], Loss: 0.1759
Epoch [2/5], Loss: 0.1241
Epoch [3/5], Loss: 0.1360
Epoch [4/5], Loss: 0.1179
Epoch [5/5], Loss: 0.2726
Test Accuracy: 96.29%
Macro Precision: 0.9640
Macro Recall: 0.9619
